In [133]:
import tensorflow as tf
import keras
from keras import layers,Model
import numpy as np

In [134]:
v = tf.Variable([4,6,7,8,8,8,9,7],dtype=tf.float32)

In [135]:
def haar1d(x):
    len = tf.shape(x)[0]
    if (len==1):
        return x[0:1]
    v_reshape = tf.reshape(x,[(len//2),2])
    v_diff = v_reshape[:,1:2] - v_reshape[:,0:1]
    v_diff = tf.reshape(v_diff,[-1])
    v_avg = tf.reduce_mean(v_reshape,axis=1)
    return tf.concat([v_diff,haar1d(v_avg)],0)

In [136]:
def haar1d_inv(x,idx):
	len = tf.shape(x)[0]
	if (idx==len):
		return x
#	print(idx,x)
	v_avg = x[-idx:]
	v_avg = tf.reshape(v_avg,[-1,1])
	v_delta = x[(len-(idx<<1)):(len-idx)] / 2
	v_delta = tf.stack([-v_delta,v_delta],axis=1)
	v_out = v_delta + v_avg
	v_out = tf.reshape(v_out,[-1])
	return haar1d_inv(tf.concat([x[:-(idx<<1)],v_out],axis=0),idx<<1)


In [137]:
encoded = haar1d(v)
decoded = haar1d_inv(encoded,1)

print(v)
print(encoded)
print(decoded)

<tf.Variable 'Variable:0' shape=(8,) dtype=float32, numpy=array([4., 6., 7., 8., 8., 8., 9., 7.], dtype=float32)>
tf.Tensor([ 2.     1.     0.    -2.     2.5    0.     1.75   7.125], shape=(8,), dtype=float32)
tf.Tensor([4. 6. 7. 8. 8. 8. 9. 7.], shape=(8,), dtype=float32)


In [138]:
def haar1d_layer(x):
    len = x.shape[1]
    if (len==1):
        return x[:,0:1]
    v_reshape = tf.reshape(x,[-1,len//2,2])
    v_diff = v_reshape[:,:,1:2]- v_reshape[:,:,0:1]
    v_diff = tf.reshape(v_diff,[-1,len//2])
    v_avg = tf.reduce_mean(v_reshape,axis=2)
    return tf.concat([v_diff,haar1d_layer(v_avg)],1)

In [139]:
def haar1d_inv_layer(x,idx):
	len = x.shape[1]
	if (idx==len):
		return x
	#print(idx,x.shape)
	v_avg = x[:,-idx:]
	v_avg = tf.reshape(v_avg,[-1,v_avg.shape[1],1])
	v_delta = x[:,(len-(idx<<1)):(len-idx)] / 2
	v_delta = tf.stack([-v_delta,v_delta],axis=2)
	v_out = v_delta + v_avg
	v_out = tf.reshape(v_out, [-1,v_out.shape[1] * v_out.shape[2]])
	return haar1d_inv_layer(tf.concat([x[:,:-(idx<<1)],v_out],axis=1),idx<<1)

In [140]:
x = layers.Input(shape=(8,))

In [144]:

y = haar1d_layer(x)
encoder = Model(x, y)
encoded = encoder.predict([[4,6,7,8,8,8,9,7],[4,6,7,8,8,8,9,7]])
z = haar1d_inv_layer(x,1)
decoder = Model(x, z)
decoded = decoder.predict(encoded)

print(encoded)
print(decoded)

1/1 [==============================] - 0s 157ms/step
[[ 2.     1.     0.    -2.     2.5    0.     1.75   7.125]
 [ 2.     1.     0.    -2.     2.5    0.     1.75   7.125]]
[[4. 6. 7. 8. 8. 8. 9. 7.]
 [4. 6. 7. 8. 8. 8. 9. 7.]]


In [142]:
o = model.predict(np.repeat([[4.,6.,7.,8.,8.,8.,9.,7.]],16384,axis=0))

512/512 [==============================] - 2s 5ms/step


In [143]:
o

array([[2.   , 1.   , 0.   , ..., 0.   , 1.75 , 7.125],
       [2.   , 1.   , 0.   , ..., 0.   , 1.75 , 7.125],
       [2.   , 1.   , 0.   , ..., 0.   , 1.75 , 7.125],
       ...,
       [2.   , 1.   , 0.   , ..., 0.   , 1.75 , 7.125],
       [2.   , 1.   , 0.   , ..., 0.   , 1.75 , 7.125],
       [2.   , 1.   , 0.   , ..., 0.   , 1.75 , 7.125]], dtype=float32)